In [3]:
import torch
import torchaudio
from transformers import Wav2Vec2Processor, Wav2Vec2Model

# 예시 로딩
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from gtts import gTTS

## gTTS를 이용한 텍스트 투 스피치 구현!!
def text_to_speech(text, out_path='temp.wav'):
    tts = gTTS(text=text, lang='ko')
    tts.save('temp.mp3')

    # mp3에서 wav로 변환
    import subprocess
    subprocess.call(['ffmpeg', '-y', '-i', 'temp.mp3', out_path])

    return out_path

In [10]:
import torch
import torchaudio
from transformers import Wav2Vec2Processor, Wav2Vec2Model

def extract_vector(audio_path):
    # load model
    processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
    model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")
    model.eval()

    # load audio
    waveform, sample_rate = torchaudio.load(audio_path)

    # resample if not 16kHz
    if sample_rate != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
        waveform = resampler(waveform)

    # 특징 추출하기
    inputs = processor(waveform.squeeze(), sampling_rate=16000, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    
    # 평균 벡터 (Global feature vector)
    return outputs.last_hidden_state.mean(dim=1)


In [23]:
## Coqui TTS 설치하
pip install TTS

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 30.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 13.6 MB/s eta 0:00:00:00:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 18.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 25.4 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 29.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 23.9 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of thinc to determ

In [24]:
import TTS

In [28]:
from TTS.api import TTS

tts = TTS()  # 빈 인스턴스 생성
models = tts.list_models()
print(models)  # 리스트로 출력해야 함

In [29]:
pip install google-cloud-texttospeech

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 20.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Not uninstalling protobuf at /usr/local/lib/python3.11/dist-packages, outside environment /home/jovyan/.venv/torch2.3.0-py3.11-cuda12.1
    Can't uninstall 'protobuf'. No files were found to uninstall.
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.64.0
    Not uninstalling grpcio at /usr/local/lib/python3.11/dist-packages, outside environment /home/jovyan/.venv/torch2.3.0-py3.11-cuda12.1
    Can't uninstall 'grpcio'. No files were found to uninstall.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.28.2 requires protobuf<5,>=3.20, but you have protobuf 6.31.0 which is incompatible.
tensorboard 2.15.0 requires protobuf<4.24,>=3.19.6, but you have protobuf 

In [22]:
import uuid
import subprocess
from pathlib import Path

class PhoneticEncoder:
    def __init__(self, tts_engine='gtts', audio_model='facebook/wav2vec2-base-960h', output_dir='tts_outputs'):
        self.tts_engine = tts_engine.lower()   ## 대소문자에 유연히 대응하기 위해 lower() 붙인다
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(parents=True, exist_ok=True)
        
        self.processor = Wav2Vec2Processor.from_pretrained(audio_model)
        self.model = Wav2Vec2Model.from_pretrained(audio_model)
        self.model.eval()

    def synthesize_tts(self, text: str, save_path=None):
        # 고유한 파일명의 생성
        uid = str(uuid.uuid4())[:8]
        mp3_path = self.output_dir / f"{uid}.mp3"
        wav_path = self.output_dir / f"{uid}.wav"
        
        if self.tts_engine == 'gtts':
            tts = gTTS(text=text, lang='ko')
            tts.save(mp3_path)   ## 추후 수정 ㄱㄱ
            subprocess.call(["ffmpeg", "-y", "-i", mp3_path, wav_path])
        elif self.tts_engine == 'clova':
            # TODO: Clova API 호출 구현
            pass
        elif self.tts_engine == 'typecast':
            # 실제 Typecast API는 공개 X -> studio 요청 복제 방식
            import requests
            
            pass
        elif self.tts_engine == 'onair':
            # TODO: 온에어스튜디오 API 호출 구현
            pass
        return save_path

    def extract_features(self, audio_path):
        waveform, sample_rate = torchaudio.load(audio_path)
        if sample_rate != 16000:
            resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
            waveform = resampler(waveform)
            
        inputs = self.processor(waveform.squeeze(), sampling_rate=16000, return_tensors="pt")
        with torch.no_grad():
            outputs = self.model(**inputs)
        return outputs.last_hidden_state.mean(dim=1)  # [1, hidden_dim]

    def encode(self, text):
        wav_path = self.synthesize_tts(text)
        feature = self.extract_features(wav_path)
        os.remove(wav_path)  # 클린업
        return feature

SyntaxError: '(' was never closed (1796138086.py, line 8)

In [11]:
text = "안녕하세요 제 이름은 이준연입니다."
audio_path = text_to_speech(text)
vector = extract_vector(audio_path)

print(vector.shape)  # torch.Size([1, 768]) 예상


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

torch.Size([1, 768])


In [13]:
clova = './tts_outputs/clova.wav'

clova_vec = extract_vector(clova)
print(clova_vec.shape)

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


torch.Size([1, 768])


In [14]:
supertone = './tts_outputs/supertone.wav'

sup_vec = extract_vector(supertone)
print(sup_vec.shape)

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


torch.Size([1, 768])


In [17]:
# 3개의 feature_vector
sup_vec
clova_vec
vector

tensor([[-4.1123e-02,  2.2890e-02, -1.7424e-01, -7.5351e-02,  1.8082e-02,
         -8.2891e-02,  8.2069e-02, -5.4274e-02,  1.2113e-01, -2.5420e-01,
         -2.8606e-02,  3.7833e-02,  6.9866e-02,  2.2229e-02, -4.9953e-02,
         -1.0299e-01, -3.9015e-01,  2.6891e-01,  2.3025e-02,  1.7120e-01,
         -1.8626e-01,  7.8245e-03,  2.3361e-01,  1.1185e-02,  3.7002e-02,
         -6.1427e-02, -1.1436e-01,  2.8381e-02, -6.1882e-02, -1.0242e-01,
          1.7282e-01, -1.1406e-02, -8.5707e-03, -7.4070e-02, -2.2447e-01,
          9.6262e-02, -8.3751e-02, -2.2628e-01, -1.0771e-01,  4.8555e-02,
         -1.6932e-01, -1.2376e-01, -6.7140e-02,  4.8048e-02, -1.0018e-01,
         -2.1151e-01, -5.6015e-02, -7.8787e-02,  4.1236e-03,  5.5842e-03,
         -1.0245e-01, -1.3810e-02,  2.2846e-01,  4.2632e-02,  1.7081e-02,
         -8.0891e-02,  4.3343e-02, -3.4825e-01, -1.3386e-01, -1.5058e-01,
         -3.0673e-02, -1.3192e-02,  1.9419e-02,  1.0631e-01, -6.2462e-03,
          7.9567e-02, -4.6762e-03, -6.

In [20]:
## 3개의 Cosine Similarity 비교하기?
from torch.nn.functional import cosine_similarity

sim_sup_gtts = cosine_similarity(vector, sup_vec)
sim_clova_sup = cosine_similarity(sup_vec, clova_vec)
sim_clova_gtts = cosine_similarity(vector, clova_vec)

print(sim_sup_gtts)
print(sim_clova_sup)
print(sim_clova_gtts)

## 결과 해석하자면.... sup_vec이랑 clova_vec 이 두가지가 유사도가 가장 높더라...

tensor([0.9616])
tensor([0.9830])
tensor([0.9609])
